# 0. Read Files
We need to be able to read/write files easily and plot some things that help us understand digital filters. 

Some of the most important files will be descriptions of filters in the form of coefficients and other parameters.

This notebook gives some useful examples of reading/writing simple files as well as parsing, plotting and transforming the values contained in the files.

## 0.a Imports

In [ ]:
import sys
import scipy as sp
import numpy as np
import matplotlib as mpl
from scipy.fftpack import fft, fftfreq
from scipy import signal

# Print lists nicely
import glob

import matplotlib.pyplot as plt
#print(plt.style.available)
plt.style.use('classic')

## 0.b Check Versions, etc

In [ ]:
print('Python: \t{:2d}.{:1d}'
      .format(sys.version_info[0], sys.version_info[1]))
print('Matplot:\t',mpl.__version__)
print('Numpy:  \t',np.__version__)
print('SciPy:  \t',sp.__version__)

# 1. Simple text files 
Simple text files are an easy way to store sets of filter parameters and other data.  The Python package "numpy" has really nice input/output features for such things.

* Read: https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html
* Write: https://numpy.org/doc/stable/reference/generated/numpy.savetxt.html

## 1.a Poles and zeroes
Poles and zeroes are one way to describe a digital filter.  

These are collections of complex values, and their relation to the "unit circle" and the "z-plane" is important. We will be talking about the unit circle and z-plane in some detail shortly.

The sets of complex values (poles, zeroes) typically describe a "transfer function" for a filter. It's called a transfer function because the action of the filter changes the input signal in some way. As a result, the _output_ of the filter is ___different___ than its _input_.

The transfer function, $F(z)$, is a function in the z-plane that is written in fraction form with numerator and denominator. It can be written in terms of multiplicative factors which are "roots" of the numerator and denominator. The $z$ value that corresponds to each root is a complex number $Z_{i}=(r+jc)$

$$F(z) = \frac{Z(z)}{P(z)} = \frac{\Pi_{i=1}^{n}(z-Z_{i})}{\Pi_{i=1}^{m}(z-P_{i})} = \frac{(z-Z_{1}) \ldots (z-Z_{n})}{(z-P_{1}) \ldots (z-P_{m})}$$

* Zeroes ($Z$) are roots of the numerator of the fraction. They make the transfer function go to zero
* Poles ($P$) are roots of the denominator of the fraction. They make the transfer function go to infinity
* Note that the factors are typically written in terms of $\frac{1}{z}=z^{-1}$ rather than $z$, but that makes the notation much harder to read

The examples below simply read flat-text file(s) that contain the complex-valued poles and zeroes.

### 1.a.1 Reads a simple text file and store the values in a list
* Zeroes are the complex-valued roots of the transfer function's __numerator__
* Poles are the complex-valued roots of the transfer function's __denominator__
* Note that they're provided in ___complex-conjugate pairs___ $(r+jc)$ and $(r-jc)$

In [ ]:
zfile = 'simple.zeroes'
zeroes = np.loadtxt(zfile, comments='#', delimiter=',', dtype=complex, unpack=False)

pfile = 'simple.poles'
poles = np.loadtxt(pfile, comments='#', delimiter=',', dtype=complex, unpack=False)

In [ ]:
def print_polezero(pp, zz):

    print('Zeroes:')
    for x in zz: print('\t{num.real:+0.04f} {num.imag:+0.04f}j'.format(num=x))
    
    print('Poles:') 
    for x in pp: print('\t{num.real:+0.04f} {num.imag:+0.04f}j'.format(num=x))

In [ ]:
print_polezero(poles, zeroes)

### 1.a.2 Read a file and dump its guts so we can see what's in it

In [ ]:
fname = 'simple.zeroes'
f = open(fname, 'r')
guts = f.read()
print(guts)
f.close()

## 1.b Filter coefficients (numerator, denominator)
Coefficients are another way to describe a digital filter. 

These are collections of (typically) real values, and their relation to the unit circle is not as direct as with poles and zeroes. However, coefficients can be converted into poles and zeroes, and vice versa.

Recall, the transfer function is represented by a complex-valued fraction in the z-plane. In addition to being written as a set of factors (poles and zeroes), this fraction can be written in terms of ___polynomials___ in $z$. 

$$F(z) = \frac{N(z)}{D(z)} 
= \frac{\sum_{i=0}^{n}a_{i}z^{-i}}{\sum_{i=0}^{m}b_{i}z^{-i}} 
= \frac{a_{0}z^{-0} a_{1}z^{-1}\ldots + a_{n}z^{-n}}{b_{0}z^{-0} b_{1}z^{-1}\ldots + b_{n}z^{-m}} $$

In this form, the filter coefficients $\{a_i\}$ and $\{b_i\}$ are the scaling factors for each power of $z$ in the numerator and denominator polynomial, rather than the __roots__ of those polynomials (as with poles and zeroes).

The examples below simply read flat-text file(s) that contain the sets of coefficients for the transfer function numerator and denominator.

### 1.b.1 Reads a simple text file and store the values in a list
* Numerator files contain the real-valued coefficients of the transfer function's __numerator__ 
* Denominator files contain the real-valued coefficients of the transfer function's __denominator__ 
* Note that they're __not__ provided in complex-conjugate pairs, but the numerator coefficients often have ___symmetric values___ around a midpoint
* Also note that in this case, the denominator has an implicit pole at the origin $(0+j0)$

In [ ]:
nfile = 'simple.numerator'
numerator = np.loadtxt(nfile, comments='#', delimiter=',', dtype=float, unpack=False)

dfile = 'simple.denominator'
denominator = np.loadtxt(dfile, comments='#', delimiter=',', dtype=float, unpack=False) 

In [ ]:
def print_numden(nn, dd):

    print('Numerator:')
    for x in nn: print('\t{num.real:+0.04f}'.format(num=x))
    
    print('Denominator:') 
    for x in dd: print('\t{num.real:+0.04f}'.format(num=x))

In [ ]:
print_numden(numerator, denominator)

### 1.b.2 Read a file and dump its guts so we can see what's in it

In [ ]:
fname = 'simple.numerator'
f = open(fname, 'r')
guts = f.read()
print(guts)
f.close()

# 2.0 Define functions for useful plots
To visualize the transfer function of a filter, we can use several techniques.
* Impulse response
* Poles and zeroes
* Frequency response (DTFT)

## 2.a Plot of impulse response
This is a simplification that only works when the denominator of the transfer function is a scalar (e.g. no poles)

In [ ]:
def plot_ImpulseResponse(numerator):
    
    fig, ax = plt.subplots(nrows=1, ncols=1)

    ax.set_title('Impulse Response (numerator)', fontsize=10)
    ax.set_axisbelow(True)
    ax.minorticks_on()

    ax.grid(which='major', linestyle='-',linewidth='0.5', color='red')
    ax.grid(which='minor', linestyle=':',linewidth='0.5', color='gray')
    ax.tick_params(which='both', top='off', bottom='off', left='off', right='off')
        
    mm = 1.1 * max(numerator)
    mn = min(1.1 * min(numerator), -0.1)
    ax.set_ylim(mn, mm)
    ax.set_xlim(-1,len(numerator))

    ax.set_xlabel('impulse response (numerator, samples)')
    ax.set_ylabel('amplitude')
    ax.axhline(linewidth=2, color='black')
    ax.axvline(linewidth=2, color='black')

    time = np.arange(0,len(numerator))
    marker, stem, base = ax.stem(time, numerator)
    stem.set_linewidth(2)

    plt.show(block=False)
    
    return

## 2.b Plot of poles and zeroes

In [ ]:
def plot_PoleZero(zeroes, poles):
    
    fig, ax = plt.subplots(nrows=1, ncols=1)

    ax.set_title('Unit Circle (z-plane)', fontsize=10)
    ax.minorticks_on()

    ax.grid(which='major', linestyle='-',linewidth='0.5', color='red')
    ax.grid(which='minor', linestyle=':',linewidth='0.5', color='gray')
    ax.tick_params(which='both',top='off', bottom='off', left='off', right='off')

    theta = np.linspace(-np.pi,np.pi,201)
    ax.plot(np.cos(theta), np.sin(theta), color='gray')

    ax.scatter(np.real(zeroes), np.imag(zeroes),facecolors='none', edgecolors='blue',marker='o')
    ax.scatter(np.real(poles), np.imag(poles), facecolors='red', marker='x')

    ax.axhline(linewidth=2, color='black')
    ax.axvline(linewidth=2, color='black')

    ax.set_ylabel('Im{z}')
    ax.set_xlabel('Re{z}')
    ax.set_aspect('equal')

    plt.show(block=False)

    return

## 2.c Plot of frequency response (DTFT)

In [ ]:
def plot_FreqResponse(zeroes, poles, srate):

    # The Python SciPy library function "freq_zpk" evaluates the set of (pole,zero) values via the DTFT
    ww, hh = signal.freqz_zpk(zeroes, poles, 1, 512, False, srate)
    fig, ax = plt.subplots(nrows=1, ncols=1)

    ax.set_title('Frequency Response (DTFT)', fontsize=10)
    ax.minorticks_on()
    ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
    ax.grid(which='minor', linestyle=':',linewidth='0.5', color='gray')
    ax.tick_params(which='both', top='off', bottom='off', left='off', right='off')
    
    ax.plot(ww,20*np.log10(abs(hh)),'b')
    
    ax.set_ylabel('Amplitude (dB)', color='b')
    ax.set_xlabel('Frequency (Hz)')
    
    ax.axhline(linewidth=2, color='black')
    ax.axvline(linewidth=2, color='black')
    
    ax2 = ax.twinx()
    angles = np.unwrap(np.angle(hh))
    ax2.plot(ww,angles,'g')
    ax2.set_ylabel('Angle (rad)', color='g')

    plt.axis('tight')
    plt.show(block=False)

    return

# 3.0 Plot the stuff we read earlier

## 3.a  Coefficients: Numerator and Denominator
First, convert the numerator/denominator coefficients $\frac{N(z)}{D(z)}=\frac{\sum a_{i}z^{-i}}{\sum b_{i}z^{-i}}$ to pole-zero format $\frac{Z(z)}{P(z)}=\frac{\Pi(z-Z_{i})}{\Pi(z-P_{i})}$

This uses the Python SciPy library function:  ___tf2zpk(N,D)___

Then use the defined plotting routines to give consistent output.

In [ ]:
nd_zeroes, nd_poles, nd_gain = signal.tf2zpk(numerator, denominator)

print_polezero(nd_poles,nd_zeroes)
print_numden(numerator,denominator)

plot_ImpulseResponse(numerator)
plot_PoleZero(nd_zeroes, nd_poles)
plot_FreqResponse(nd_zeroes, nd_poles, 2000)

## 3.b Poles and Zeroes
First, convert the pole/zero values $\frac{Z(z)}{P(z)}=\frac{\Pi(z-Z_{i})}{\Pi(z-P_{i})}$ to numerator/denominator format $\frac{N(z)}{D(z)}=\frac{\sum a_{i}z^{-i}}{\sum b_{i}z^{-i}}$

This uses the Python SciPy library function:  ___zpk2tf(Z,P)___

Then use the defined plotting routines to give consistent output.

In [ ]:
pz_numerator, pz_denominator = signal.zpk2tf(zeroes, poles, 1)

print_numden(pz_numerator,pz_denominator)

# Skipping impulse response here because the presence of a denominator makes the process more complicated ...
#plot_ImpulseResponse(pz_numerator)
plot_PoleZero(zeroes, poles)
plot_FreqResponse(zeroes, poles, 2000)

# Questions

### 1. Different Filters
The two sets of files that I provided ... _(simple.poles, simple.zeroes)_ and _(simple.numerator, simple.denominator)_ ... describe two filters with ___different___ transfer functions. You can see this from the previous graphs, etc. where the plots for the set of _(pole,zero)_ files are very different from the plots for the set of _(numerator,denominator)_ files.

* How are the two filters different?
* What is your interpretation of the spectral plots?
* What's the relationship between the spectral plots in each case and the "unit circle"?
* Do the filters only change the frequency content, or do they do something else as well?

### 2. Roll Your Own
You should be able to create simple files of your own following the structure of the _(simple.poles, simple.zeroes)_ and _(simple.numerator, simple.denominator)_ files. Can you create sets of simple pole/zero and numerator/denominator values that describe a digital filter with ___the same___ transfer function?  Don't modify the files I provided.  Create new ones with your own values.

* Create one set of files containing complex pole/zero pairs
* Create another set of files containing numerator/denominator coefficients
* Plot their transfer functions, etc. ___using side-by-side subplots___ to show that they perform equivalently

### 3. Impulse Response
Notice the caveats in the "plot_ImpulseResponse" function, and the fact that we didn't use this in the case of one of the provided filters (the one with an actual _denominator polynomial_ .. the one with _poles_).

* Describe what's going on in the function "plot_ImpulseResponse".  What is actually being plotted?
* Why wouldn't this approach work in the case where the transfer function has poles in the denominator?

# 1

The sets of files given show two separate digital filters that do different kinds of transfer work. One set shows filters based on their z-plane poles and zeros, while the other set uses the numerator and denominator polynomials' coefficients in the z-domain to describe how the filters behave.

We can see where the poles and zeroes are in the z-plane by looking at the spectral plots that are made from poles and zeroes. These positions have a direct effect on the frequency response of the filter. The numerator and denominator factors, on the other hand, show how the filter changes the frequency parts of the input signal in spectral charts.


For both types of filters, the connection between the spectral plots and the unit circle in the z-plane is important. For filters with poles and zeroes, the stability and frequency response are based on where the unit circle is in relation to the poles and zeroes. For filters with numerator and denominator factors, on the other hand, frequencies at the unit circle are equal to the Nyquist frequency and change the frequency response of the filter.

The main job of both types of filters is to change the frequency content of the input stream. But, based on how they are built and what they're made of, they may also add phase shifts and intensity changes. These filters change the input sound based on their transfer functions by cutting down on some frequencies while letting others through.

# 2

In [ ]:
import numpy as np

zeroes_complex = [0.2*np.exp(1j*np.pi/4), 0.2*np.exp(-1j*np.pi/4)]
poles_complex = [0.9*np.exp(1j*np.pi/6), 0.9*np.exp(-1j*np.pi/6)]

np.savetxt('complex.zeroes', zeroes_complex, delimiter=',', fmt='%0.4f%+0.4fj')
np.savetxt('complex.poles', poles_complex, delimiter=',', fmt='%0.4f%+0.4fj')

numerator_coeffs = [1]
denominator_coeffs = [1, -1.4142, 1]

np.savetxt('complex.numerator', numerator_coeffs, delimiter=',', fmt='%0.4f')
np.savetxt('complex.denominator', denominator_coeffs, delimiter=',', fmt='%0.4f')

import matplotlib.pyplot as plt
from scipy import signal

zeroes_complex_loaded = np.loadtxt('complex.zeroes', delimiter=',', dtype=complex)
poles_complex_loaded = np.loadtxt('complex.poles', delimiter=',', dtype=complex)
numerator_coeffs_loaded = np.loadtxt('complex.numerator', delimiter=',')
denominator_coeffs_loaded = np.loadtxt('complex.denominator', delimiter=',')

w, h_complex = signal.freqz_zpk(zeroes_complex_loaded, poles_complex_loaded, 1)
w, h_coeffs = signal.freqz(numerator_coeffs_loaded, denominator_coeffs_loaded)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(w, 20 * np.log10(abs(h_complex)), 'b', label='Complex Pole/Zero')
plt.plot(w, 20 * np.log10(abs(h_coeffs)), 'r--', label='Numerator/Denominator')
plt.title('Frequency Response')
plt.xlabel('Frequency (rad/sample)')
plt.ylabel('Amplitude (dB)')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(w, np.angle(h_complex), 'b', label='Complex Pole/Zero')
plt.plot(w, np.angle(h_coeffs), 'r--', label='Numerator/Denominator')
plt.title('Phase Response')
plt.xlabel('Frequency (rad/sample)')
plt.ylabel('Phase (radians)')
plt.legend()

plt.tight_layout()
plt.show()


# 3
A method called "plot_ImpulseResponse" shows the impulse response of a digital filter over time in the form of an impulse input signal. The function shows how the filter works in the time domain by showing the numerator coefficients, which are the impulse response, against time indices. But this method depends on the system being stable and not having any poles in the base of the transfer function.

If the transfer function has poles in the minimum, the function's method might not work because the system might become unstable. Filters with poles in the denominator can have impulse responses that are either infinite or not defined at all. This makes plotting the impulse response useless and doesn't tell you much about how the filter works. Because of this, you should be careful when using the "plot_ImpulseResponse" method, especially with filters that have complicated transfer functions.